In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SaveMode

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path

println("Initializing Spark context...")

Intitializing Scala interpreter ...

Spark Web UI available at http://SIddhartha:4040
SparkContext available as 'sc' (version = 3.0.3, master = local[*], app id = local-1653472681972)
SparkSession available as 'spark'


Initializing Spark context...


import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SaveMode
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path


In [2]:
val df = spark.read.option("header",true).option("inferSchema",true).csv("Data/OfficeDataProject.csv")
df.show()

print("\n"+"--"*30+"\n")

print(df.printSchema())

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

df: org.apache.spark.sql.DataFrame = [employee_id: int, employee_name: string ... 5 more fields]


In [3]:
// COMMAND ----------
 
//UDF -> User Defined Functions(Custom Requirements)
//#Simple UDF
//#increase salary for each employee by state
//#if state is NY -> salary by 10% and bonus by 5%
//#if state is CA -> salary by 12% and bonus by 3%
def total_salary(state:String ,salary: Int,bonus: Int): Double = {
    var demo = 1.0
    if (state == "NY"){
        demo = (salary * 0.10)
        demo =  demo + bonus * 0.05
        }
    else if (state == "CA"){
        demo = (salary * 0.12)
        demo =  demo + bonus * 0.03
        }
    return demo
 
 }

print("using above functions",total_salary("CA",1000,100))

(using above functions,123.0)

total_salary: (state: String, salary: Int, bonus: Int)Double


#---------------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------------


In [4]:
 
var sidudf = udf((x: String,y:Int, z:Int) => total_salary(x,y,z)) 
// 
df.withColumn("increment",sidudf(df("state"),df("salary"),df("bonus"))).show();
 
//
//  val featureHashUDF = udf((dim: String, value: String) => WalLogAgg.toFeatureHash(dim, value))
//
//    filteredDict.withColumn("featureHash", featureHashUDF(col("dim"), col("value")))
 //     .select("featureHash")

+-----------+-------------------+----------+-----+------+---+-----+------------------+
|employee_id|      employee_name|department|state|salary|age|bonus|         increment|
+-----------+-------------------+----------+-----+------+---+-----+------------------+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|            752.01|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|               1.0|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|               1.0|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|               1.0|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|            724.02|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|373.49999999999994|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|            919.14|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|               1.0|
|       1008|  Recalde Kensinger|  Accounts

sidudf: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$3405/720466668@4510ffff,DoubleType,List(Some(class[value[0]: string]), Some(class[value[0]: int]), Some(class[value[0]: int])),None,false,true)


In [5]:
df.cache()

res4: df.type = [employee_id: int, employee_name: string ... 5 more fields]


#---------------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------------


In [6]:
var spark = SparkSession.builder.appName("UDF").getOrCreate()
 

var df = spark.read.option("header",true).option("inferSchema",true).csv("Data/StudentData.csv")
df.show()
 

+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|  2984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|
| 28|  Male|  Celeste Lollis|   MVC| 81492|   64|Nicole Harwood_Cl...|
| 29|  Male|  Cordie Harnois|   OOP| 92882|   51|Judie Chipps_Clem...|
| 29|Female|       Kena Wild|   DSA|102285|   35|Dustin Feagins_Ma...|
| 29| 

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@15784d84
df: org.apache.spark.sql.DataFrame = [age: int, gender: string ... 5 more fields]


In [7]:
//COMMAND ----------
 
println(df)
print(df("age").getClass)
 
//COMMAND ----------
 
//df.collect()
 
// COMMAND ----------
 


[age: int, gender: string ... 5 more fields]
class org.apache.spark.sql.Column

In [9]:
df.filter("age==28").show()

+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|  2984|   59|Annika Hoffman_Na...|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|
| 28|  Male|  Celeste Lollis|   MVC| 81492|   64|Nicole Harwood_Cl...|
| 28|Female|  Latia Vanhoose|    DB|122502|   27|Latia Vanhoose_Mi...|
| 28|Female|Mickey Cortright|    DB|192537|   62|Ernest Rossbach_M...|
| 28|Female|       Kena Wild| Cloud|221750|   60|Mitzi Seldon_Jenn...|
| 28|Female|    Jc Andrepont|   DSA|232060|   58|Billi Clore_Abram...|
| 28|F

In [11]:
// COMMAND ----------
val df = spark.read.option("header", true).option("inferSchema", true).csv("Data/StudentData.csv")
var rdd = df.rdd
println(rdd.getClass)

//    val filt = lines.filter(text=>(text.toString.contains(tt => tt in (startdate until enddate))))


rdd.filter(age =>(df("age") == 28)).collect()

 
// COMMAND ----------
 
rdd.take(1)
 
// COMMAND ----------
 
//rdd.filter(df("age") == 28).collect()


class org.apache.spark.rdd.MapPartitionsRDD


org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 13, SIddhartha, executor driver): org.apache.spark.SparkException: Dataset transformations and actions can only be invoked by the driver, not inside of other Dataset transformations; for example, dataset1.map(x => dataset2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the dataset1.map transformation. For more information, see SPARK-28702.